<a href="https://colab.research.google.com/github/hafezgh/NBML-BCI-Competetion-2022/blob/main/NBML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q mne

     |████████████████████████████████| 7.5 MB 4.5 MB/s 


In [ ]:
from collections import Counter
import numpy as np
import plotly.express as px
import mne
from mne.decoding import CSP
from sklearn.metrics import confusion_matrix, plot_confusion_matrix, classification_report, f1_score
from scipy import signal
from sklearn.decomposition import PCA
import scipy.io
from sklearn.preprocessing import normalize, StandardScaler, MinMaxScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn import svm
import copy
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE, RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler, InstanceHardnessThreshold, NearMiss
from tqdm.notebook import tqdm

from scipy.fft import fft, fftfreq
import matplotlib.pyplot as plt
from scipy import signal
from scipy.signal import filtfilt
from scipy.signal import butter
import pandas as pd 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
SAMPLE_RATE = 125
DURATION = 2
nyquist = SAMPLE_RATE/2
# Number of samples in normalized_tone
N = SAMPLE_RATE * DURATION

In [ ]:
def butter_bandpass_filter(data, lowcut, highcut, fs, order):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    y = filtfilt(b, a, data)
    return y

In [ ]:
## Read Data
chans = scipy.io.loadmat(f'/content/drive/MyDrive/NBML BCI 2022/New_Data/BCI5_Data/Channels.mat')
all_subjs_dict = dict()
n_subjects = 15
for i in range(n_subjects):
    data = scipy.io.loadmat(f'/content/drive/MyDrive/NBML BCI 2022/New_Data/BCI5_Data/Subject_{i+1}.mat')
    all_subjs_dict[i] = data
## Load Data
n_trials = 18
n_trials_test = 4
n_channels = 16 # time not included
n_windows = 5
timepoints = 1250
X = np.zeros((n_subjects*n_trials, timepoints, n_channels))
y = np.zeros((n_subjects*n_trials, n_windows, 1))

Xt = np.zeros((n_subjects*n_trials_test, timepoints, n_channels))
yt = np.zeros((n_subjects*n_trials_test, n_windows, 1))

# BC: Binary Classification
n_segments = 5
n_segments_test = 5
timepoints_bc = 250
X_binary = np.zeros((n_subjects*n_trials*n_segments, timepoints_bc, n_channels))
y_binary = np.zeros((n_subjects*n_trials*n_segments, 1))

X_binary_test = np.zeros((n_subjects*n_trials_test*n_segments, timepoints_bc, n_channels))
y_binary_test = np.zeros((n_subjects*n_trials_test*n_segments, 1))

idx=0
idxt=0
for s in range(n_subjects):
    data = all_subjs_dict[s]
    for i in range(n_trials):
        n_timepoints = min(data["TrainData"][0][i][0][0][0].shape[0], 1250) # handle shape mismatch (some trials do not have exactly 1250 timepoints -> prune or zero-pad)
        X[s*n_trials+i, :n_timepoints, :] = np.array(data["TrainData"][0][i][0][0][0][:n_timepoints,:n_channels])
        y[s*n_trials+i, :, :] = np.array(data["TrainData"][0][i][0][0][1])
        for seg in range(n_segments):
            X_binary[idx, :, :] = X[s*n_trials+i, seg*timepoints_bc:(seg+1)*timepoints_bc, :]
            y_binary[idx,0] = y[s*n_trials+i, seg, :]
            idx+=1

n_subjects_test = 10 
for s in range(n_subjects_test):
    data = all_subjs_dict[s]
    for i in range(n_trials_test):
        n_timepoints = min(data["TestData"][0][i][0][0][0].shape[0], 1250) # handle shape mismatch (some trials do not have exactly 1250 timepoints -> prune or zero-pad)
        Xt[s*n_trials_test+i, :n_timepoints, :] = np.array(data["TestData"][0][i][0][0][0][:n_timepoints,:n_channels])
        for seg in range(n_segments):
            X_binary_test[idxt, :, :] = X[s*n_trials_test+i, seg*timepoints_bc:(seg+1)*timepoints_bc, :]
            idxt+=1

print(f"data shape: {X.shape}, label shape: {y.shape}")
print(f"binary classification data shape: {X_binary.shape}, label shape: {y_binary.shape}")
print(f"test data shape: {Xt.shape}, label shape: {yt.shape}")
print(f"binary classification test data shape: {X_binary_test.shape}, label shape: {y_binary_test.shape}")


data shape: (270, 1250, 16), label shape: (270, 5, 1)
binary classification data shape: (1350, 250, 16), label shape: (1350, 1)
test data shape: (60, 1250, 16), label shape: (60, 5, 1)
binary classification test data shape: (300, 250, 16), label shape: (300, 1)


In [ ]:
def train_validate(X_bc, y_bc, X_bct, y_bct,
                   sampling= RandomOverSampler(), n_comps= 6, clf = LinearDiscriminantAnalysis(), clf_final = LinearDiscriminantAnalysis(),
                   subject= None, plot_results= True, synchronization=True, scaler= StandardScaler() ,chans_to_remove = ['O2','T8'] , seed= 0,
                   filter_bands = [(1,4),(4,8) ,(8, 13),(23,32)], ):
    
    np.random.seed(seed)
    # Channel selection
    chans_dict = dict()
    for i in range(len(chans['Channels'][0])):
        chans_dict[chans['Channels'][0][i][0]] = i
    chans_to_remove_idx = [chans_dict[chan] for chan in chans_to_remove]  
    chans_to_keep = list(Counter(list(range(16))) - Counter(chans_to_remove_idx))
    ### keep only the useful channels
    X_bc = X_bc[:,:,chans_to_keep]
    X_bct = X_bct[:,:,chans_to_keep]
    ### Synchronization
    if synchronization:
        for sub in range(n_subjects):
            x_subj = X_bc[sub*90:(sub+1)*90,...]
            x_subjt = X_bct[sub*90:(sub+1)*90,...]
            y_subj = y_bc[sub*90:(sub+1)*90,...]
            idx = []
            for i in range(y_subj.shape[0]):
                if y_subj[i] == 0:
                    idx.append(i)
            x_subj0 = x_subj[idx,...]
            x_subj0_mean = np.mean(x_subj, axis=0)
            x_subj -= x_subj0_mean
            x_subjt -= x_subj0_mean
            X_bc[sub*90:(sub+1)*90,...] = x_subj
            X_bct[sub*90:(sub+1)*90,...] = x_subjt

    ### Normalization
    if scaler is not None:
        n_samples, tp, nchannels = X_bc.shape
        scaled_X = scaler.fit_transform(X_bc.reshape([n_samples, -1]))
        n_samples_t, tp_t, nchannels_t = X_bct.shape
        scaled_Xt = scaler.fit_transform(X_bct.reshape([n_samples_t, -1]))

        X_bc = scaled_X.reshape([n_samples, tp, nchannels])
        X_bct = scaled_Xt.reshape([n_samples_t, tp_t, nchannels_t])
    ### filtering
    X_bc_bands = np.zeros((X_bc.shape[0], len(filter_bands), X_bc.shape[1], X_bc.shape[2]))
    X_bct_bands = np.zeros((X_bct.shape[0], len(filter_bands), X_bct.shape[1], X_bct.shape[2]))

    for i in range(len(filter_bands)):
        x = copy.deepcopy(X_bc)
        for chan in range(X_bc.shape[2]):
            for trial in range(X_bc.shape[0]):
                X_bc_bands[trial,i,:,chan] = butter_bandpass_filter(X_bc[trial,:,chan], filter_bands[i][0], filter_bands[i][1], SAMPLE_RATE, 5)
            for trial in range(X_bct.shape[0]):
                X_bct_bands[trial,i,:,chan] = butter_bandpass_filter(X_bct[trial,:,chan], filter_bands[i][0], filter_bands[i][1], SAMPLE_RATE, 5)
    
    x = X_bc[4,:,12]
    x_filt = butter_bandpass_filter(x, 4, 40, SAMPLE_RATE, 5)
    yf = fft(x_filt)
    xf = fftfreq(N, 1 / SAMPLE_RATE)
    # Train-test Split
    ## Subject-dependent data
    X_bc_subj = dict()
    y_bc_subj = dict()
    X_bc_subj_bands = dict()

    X_bct_subj = dict()
    y_bct_subj = dict()
    X_bct_subj_bands = dict()

    for i in range(n_subjects):
        X_bc_subj[i] = X_bc[i*90:(i+1)*90,...]
        y_bc_subj[i] = y_bc[i*90:(i+1)*90,...]
        X_bc_subj_bands[i] = X_bc_bands[i*90:(i+1)*90,...]
        X_bct_subj[i] = X_bct[i*20:(i+1)*20,...]
        y_bct_subj[i] = y_bct[i*20:(i+1)*20,...]
        X_bct_subj_bands[i] = X_bct_bands[i*20:(i+1)*20,...]

    if subject is None: #SI 
        X = X_bc_bands
        y = y_bc

    else: #SD
        X = X_bc_subj_bands[subject]
        y = y_bc_subj[subject]

    ### stratified sampling to address class imbalance
    X_train, X_val, y_train, y_val = train_test_split(X, y, shuffle=True, test_size=0.2, random_state=2045, stratify=y)
    orig_shape = X_train.shape
    print("Before oversampling:", X_train.shape, X_val.shape, y_train.shape, y_val.shape)

    arr = X_train.copy()
    arr = np.reshape(arr, (arr.shape[0], arr.shape[1]*arr.shape[2]*arr.shape[3]))
    X_train_res, y_train_res = sampling.fit_resample(arr, y_train)
    X_train_res = np.reshape(X_train_res, (X_train_res.shape[0], orig_shape[1], orig_shape[2], orig_shape[3]))

    X_train = X_train_res.copy()
    y_train = np.reshape(y_train_res, (y_train_res.shape[0],1))
    print("After oversampling:", X_train.shape, X_val.shape, y_train.shape, y_val.shape)

    # SBCSP
    features = np.zeros((X_train.shape[0], len(filter_bands)))
    features_val = np.zeros((X_val.shape[0], len(filter_bands)))

    for i in range(len(filter_bands)):
        csp = CSP(n_components=n_comps, reg=1e-2)
        x = X_train[:,i,:,:].reshape(X_train.shape[0], X_train.shape[2], X_train.shape[3])
        x_val = X_val[:,i,:,:].reshape(X_val.shape[0], X_val.shape[2], X_val.shape[3])
        components = csp.fit_transform(x, y_train.ravel())
        components_val = csp.transform(x_val)
        clf.fit(components, y_train.ravel())
        features[:,i] = clf.predict(components).ravel()
        features_val[:,i] = clf.predict(components_val).ravel()
    

    clf_final.fit(features, y_train)
    yhat_train = clf_final.predict(features)
    yhat = clf_final.predict(features_val)
    yhat_scores = clf_final.predict_proba(features_val)

    if plot_results:
        print("train")
        print(classification_report(y_train, yhat_train))
        print("val")
        print(classification_report(y_val, yhat))

        plot_confusion_matrix(clf, features, y_train)
        plt.title("train")
        plot_confusion_matrix(clf, features_val, y_val)
        plt.title('val')
        plt.show()

    return f1_score(y_val, yhat, average= "macro"), scaler, csp, clf, clf_final

In [ ]:
grid_search= {}
for sampling in [RandomOverSampler(random_state=42), RandomUnderSampler(random_state=42)]:
    for clf in [svm.SVC(probability=True), LinearDiscriminantAnalysis(), KNeighborsClassifier(n_neighbors=3)]:
        for n_comps in [10,12]:
            for scaler in [StandardScaler(), MinMaxScaler()]:
                for synchronization in [True, False]:
                    f1 = train_validate(X_binary, y_binary, X_binary_test, y_binary_test,
                                    sampling= sampling, n_comps= n_comps, clf =clf, 
                                    synchronization=synchronization, scaler= scaler,
                                    filter_bands = [(1,4),(4,8),(8, 13),(23,32)],
                                    plot_results= False,subject= None, chans_to_remove = ['O2','T8'], seed= 0,)
                    grid_search[(sampling, clf, n_comps,scaler,synchronization)] = f1

In [ ]:
for m, r in sorted(grid_search.items(), key = lambda x: x[1])[::-1]:
    print(m, ':', r)

(RandomUnderSampler(random_state=42), SVC(probability=True), 10, StandardScaler(), True) : 0.5652173913043479
(RandomOverSampler(random_state=42), LinearDiscriminantAnalysis(), 12, StandardScaler(), False) : 0.5393398516963055
(RandomOverSampler(random_state=42), LinearDiscriminantAnalysis(), 10, MinMaxScaler(), False) : 0.5301587301587302
(RandomOverSampler(random_state=42), SVC(probability=True), 12, StandardScaler(), True) : 0.5255212077641984
(RandomOverSampler(random_state=42), KNeighborsClassifier(n_neighbors=3), 12, StandardScaler(), False) : 0.5234826943818023
(RandomOverSampler(random_state=42), SVC(probability=True), 10, StandardScaler(), True) : 0.5218750000000001
(RandomOverSampler(random_state=42), LinearDiscriminantAnalysis(), 10, StandardScaler(), True) : 0.5157162543473981
(RandomUnderSampler(random_state=42), KNeighborsClassifier(n_neighbors=3), 10, StandardScaler(), False) : 0.5148663101604278
(RandomUnderSampler(random_state=42), LinearDiscriminantAnalysis(), 10, Sta

In [ ]:
f1_scores = np.zeros((5,10))
for seed in tqdm(range(10)):
    for i, (sampling, clf, n_comps, scaler, synchronization) in enumerate([
            (RandomUnderSampler(random_state=seed), svm.SVC(probability=True), 10, StandardScaler(), True),
            (RandomOverSampler(random_state=seed), LinearDiscriminantAnalysis(), 12, StandardScaler(), False),
            (RandomOverSampler(random_state=seed), LinearDiscriminantAnalysis(), 10, MinMaxScaler(), False),
            (RandomOverSampler(random_state=seed), svm.SVC(probability=True), 12, StandardScaler(), True),
            (RandomOverSampler(random_state=seed), KNeighborsClassifier(n_neighbors=3), 12, StandardScaler(), False),
    ]): 
        f1_scores[i,seed] = train_validate(X_binary, y_binary, X_binary_test, y_binary_test,
                                    sampling= sampling, n_comps= n_comps, clf =clf, 
                                    synchronization=synchronization, scaler= scaler,
                                    filter_bands = [(1,4),(4,8),(8, 13),(23,32)],
                                    plot_results= False,subject= None, chans_to_remove = ['O2','T8'], seed= seed,)

  0%|          | 0/10 [00:00<?, ?it/s]

Before oversampling: (720, 4, 250, 14) (180, 4, 250, 14) (720, 1) (180, 1)
After oversampling: (206, 4, 250, 14) (180, 4, 250, 14) (206, 1) (180, 1)
Computing rank from data with rank=None
    Using tolerance 8.9 (2.2e-16 eps * 250 dim * 1.6e+14  max singular value)
    Estimated rank (mag): 147
    MAG: rank 147 computed from 250 data channels with 0 projectors
    Setting small MAG eigenvalues to zero (without PCA)
Reducing data rank from 250 -> 147
Estimating covariance using SHRINKAGE
Done.
Computing rank from data with rank=None
    Using tolerance 8.8 (2.2e-16 eps * 250 dim * 1.6e+14  max singular value)
    Estimated rank (mag): 147
    MAG: rank 147 computed from 250 data channels with 0 projectors
    Setting small MAG eigenvalues to zero (without PCA)
Reducing data rank from 250 -> 147
Estimating covariance using SHRINKAGE
Done.
Computing rank from data with rank=None
    Using tolerance 3.7 (2.2e-16 eps * 250 dim * 6.7e+13  max singular value)
    Estimated rank (mag): 174
 

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Before oversampling: (720, 4, 250, 14) (180, 4, 250, 14) (720, 1) (180, 1)
After oversampling: (1234, 4, 250, 14) (180, 4, 250, 14) (1234, 1) (180, 1)
Computing rank from data with rank=None
    Using tolerance 22 (2.2e-16 eps * 250 dim * 4e+14  max singular value)
    Estimated rank (mag): 148
    MAG: rank 148 computed from 250 data channels with 0 projectors
    Setting small MAG eigenvalues to zero (without PCA)
Reducing data rank from 250 -> 148
Estimating covariance using SHRINKAGE
Done.
Computing rank from data with rank=None
    Using tolerance 21 (2.2e-16 eps * 250 dim * 3.7e+14  max singular value)
    Estimated rank (mag): 147
    MAG: rank 147 computed from 250 data channels with 0 projectors
    Setting small MAG eigenvalues to zero (without PCA)
Reducing data rank from 250 -> 147
Estimating covariance using SHRINKAGE
Done.
Computing rank from data with rank=None
    Using tolerance 8.9 (2.2e-16 eps * 250 dim * 1.6e+14  max singular value)
    Estimated rank (mag): 176
   

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


...

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Before oversampling: (720, 4, 250, 14) (180, 4, 250, 14) (720, 1) (180, 1)
After oversampling: (1234, 4, 250, 14) (180, 4, 250, 14) (1234, 1) (180, 1)
Computing rank from data with rank=None
    Using tolerance 22 (2.2e-16 eps * 250 dim * 4e+14  max singular value)
    Estimated rank (mag): 148
    MAG: rank 148 computed from 250 data channels with 0 projectors
    Setting small MAG eigenvalues to zero (without PCA)
Reducing data rank from 250 -> 148
Estimating covariance using SHRINKAGE
Done.
Computing rank from data with rank=None
    Using tolerance 22 (2.2e-16 eps * 250 dim * 3.9e+14  max singular value)
    Estimated rank (mag): 146
    MAG: rank 146 computed from 250 data channels with 0 projectors
    Setting small MAG eigenvalues to zero (without PCA)
Reducing data rank from 250 -> 146
Estimating covariance using SHRINKAGE
Done.
Computing rank from data with rank=None
    Using tolerance 8.9 (2.2e-16 eps * 250 dim * 1.6e+14  max singular value)
    Estimated rank (mag): 176
   

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Before oversampling: (720, 4, 250, 14) (180, 4, 250, 14) (720, 1) (180, 1)
After oversampling: (1234, 4, 250, 14) (180, 4, 250, 14) (1234, 1) (180, 1)
Computing rank from data with rank=None
    Using tolerance 22 (2.2e-16 eps * 250 dim * 4e+14  max singular value)
    Estimated rank (mag): 148
    MAG: rank 148 computed from 250 data channels with 0 projectors
    Setting small MAG eigenvalues to zero (without PCA)
Reducing data rank from 250 -> 148
Estimating covariance using SHRINKAGE
Done.
Computing rank from data with rank=None
    Using tolerance 22 (2.2e-16 eps * 250 dim * 3.9e+14  max singular value)
    Estimated rank (mag): 146
    MAG: rank 146 computed from 250 data channels with 0 projectors
    Setting small MAG eigenvalues to zero (without PCA)
Reducing data rank from 250 -> 146
Estimating covariance using SHRINKAGE
Done.
Computing rank from data with rank=None
    Using tolerance 8.9 (2.2e-16 eps * 250 dim * 1.6e+14  max singular value)
    Estimated rank (mag): 176
   

/usr/local/lib/python3.7/dist-packages/sklearn/neighbors/_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


In [ ]:
print(f1_scores.mean(1))

[0.45559222 0.48970872 0.50133792 0.48367163 0.46769877]


## Further Tests
2.   6 classes[train/val, just val]

# Subject Dependent 

In [ ]:
f1_scores= [] 
for i in range(n_subjects):
    for (sampling, clf, n_comps, scaler, Synchronization) in [(RandomOverSampler(random_state=seed), LinearDiscriminantAnalysis(), 10, MinMaxScaler(), False)]: 
        f1_scores.append(train_validate(X_binary, y_binary, X_binary_test, X_binary_test,
                    sampling= sampling, n_comps= n_comps, clf =clf, 
                    synchronization=synchronization, scaler= scaler,
                    filter_bands = [(1,4),(4,8),(8, 13),(23,32)],
                    plot_results= False, subject= i, chans_to_remove = ['O2','T8'], seed= 1))


Before oversampling: (72, 4, 250, 14) (18, 4, 250, 14) (72, 1) (18, 1)
After oversampling: (124, 4, 250, 14) (18, 4, 250, 14) (124, 1) (18, 1)
Computing rank from data with rank=None
    Using tolerance 1.1 (2.2e-16 eps * 250 dim * 2e+13  max singular value)
    Estimated rank (mag): 141
    MAG: rank 141 computed from 250 data channels with 0 projectors
    Setting small MAG eigenvalues to zero (without PCA)
Reducing data rank from 250 -> 141
Estimating covariance using SHRINKAGE
Done.
Computing rank from data with rank=None
    Using tolerance 1.4 (2.2e-16 eps * 250 dim * 2.4e+13  max singular value)
    Estimated rank (mag): 127
    MAG: rank 127 computed from 250 data channels with 0 projectors
    Setting small MAG eigenvalues to zero (without PCA)
Reducing data rank from 250 -> 127
Estimating covariance using SHRINKAGE
Done.


LinAlgError: ignored

In [ ]:
print("F1 scores")
for i in range(n_subjects):
    print(f"Subject {i+1}: {f1_scores[i]}")

#Test

In [ ]:
sampling, clf, n_comps, scaler, synchronization = (RandomOverSampler(random_state=0), LinearDiscriminantAnalysis(), 10, MinMaxScaler(), False)
f1score, trained_scaler, trained_csp, trained_clf, trained_clf_final  = train_validate(X_binary, y_binary, X_binary_test, y_binary_test,
                                    sampling= sampling, n_comps= n_comps, clf =clf, clf_final = LinearDiscriminantAnalysis(),
                                    synchronization=synchronization, scaler= scaler,
                                    filter_bands = [(1,4),(4,8),(8, 13),(23,32)],
                                    plot_results= False,subject= None, chans_to_remove = ['O2','T8'], seed= 0)

Before oversampling: (1080, 4, 250, 14) (270, 4, 250, 14) (1080, 1) (270, 1)
After oversampling: (1848, 4, 250, 14) (270, 4, 250, 14) (1848, 1) (270, 1)
Computing rank from data with rank=None
    Using tolerance 4.1 (2.2e-16 eps * 250 dim * 7.4e+13  max singular value)
    Estimated rank (mag): 146
    MAG: rank 146 computed from 250 data channels with 0 projectors
    Setting small MAG eigenvalues to zero (without PCA)
Reducing data rank from 250 -> 146
Estimating covariance using SHRINKAGE
Done.
Computing rank from data with rank=None
    Using tolerance 4.2 (2.2e-16 eps * 250 dim * 7.5e+13  max singular value)
    Estimated rank (mag): 145
    MAG: rank 145 computed from 250 data channels with 0 projectors
    Setting small MAG eigenvalues to zero (without PCA)
Reducing data rank from 250 -> 145
Estimating covariance using SHRINKAGE
Done.
Computing rank from data with rank=None
    Using tolerance 1.7 (2.2e-16 eps * 250 dim * 3e+13  max singular value)
    Estimated rank (mag): 174

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [ ]:
def Test(X_bc, scaler, csp, clf, final_clf,
         n_comps= 6,  subject= None, plot_results= True, synchronization=True, chans_to_remove = ['O2','T8'] , seed= 0,
                   filter_bands = [(1,4),(4,8) ,(8, 13),(23,32)], ):
    
    np.random.seed(seed)
    # Channel selection
    chans_dict = dict()
    for i in range(len(chans['Channels'][0])):
        chans_dict[chans['Channels'][0][i][0]] = i
    chans_to_remove_idx = [chans_dict[chan] for chan in chans_to_remove]  
    chans_to_keep = list(Counter(list(range(16))) - Counter(chans_to_remove_idx))
    ### keep only the useful channels
    X_bc = X_bc[:,:,chans_to_keep]


    ### Normalization
    if scaler is not None:
        n_samples, tp, nchannels = X_bc.shape
        scaled_X = scaler.fit_transform(X_bc.reshape([n_samples, -1]))
        X_bc = scaled_X.reshape([n_samples, tp, nchannels])
    ### filtering
    X_bc_bands = np.zeros((X_bc.shape[0], len(filter_bands), X_bc.shape[1], X_bc.shape[2]))

    for i in range(len(filter_bands)):
        x = copy.deepcopy(X_bc)
        for chan in range(X_bc.shape[2]):
            for trial in range(X_bc.shape[0]):
                X_bc_bands[trial,i,:,chan] = butter_bandpass_filter(X_bc[trial,:,chan], filter_bands[i][0], filter_bands[i][1], SAMPLE_RATE, 5)
  
    x = X_bc[4,:,12]
    x_filt = butter_bandpass_filter(x, 4, 40, SAMPLE_RATE, 5)
    yf = fft(x_filt)
    xf = fftfreq(N, 1 / SAMPLE_RATE)
    # Train-test Split
    ## Subject-dependent data
    X_bc_subj = dict()
    y_bc_subj = dict()
    X_bc_subj_bands = dict()

    for i in range(n_subjects):
        X_bc_subj[i] = X_bc[i*90:(i+1)*90,...]
        X_bc_subj_bands[i] = X_bc_bands[i*90:(i+1)*90,...]

    if subject is None: #SI 
        X = X_bc_bands

    else: #SD
        X = X_bc_subj_bands[subject]

    # SBCSP
    features = np.zeros((X.shape[0], len(filter_bands)))

    for i in range(len(filter_bands)):
        x = X[:,i,:,:].reshape(X.shape[0], X.shape[2], X.shape[3])
        components = csp.transform(x)
        features[:,i] = clf.predict(components).ravel()
    
    yhat = final_clf.predict(features)
    prob = final_clf.predict_proba(features)[:,1]


    return yhat, prob

In [ ]:
y_hat, prob = Test(X_binary, trained_scaler, trained_csp, trained_clf,trained_clf_final,
    n_comps= n_comps,  subject= None, plot_results= False, 
    synchronization=synchronization, chans_to_remove = ['O2','T8'] , seed= 0,
    filter_bands = [(1,4),(4,8) ,(8, 13),(23,32)], 
     )